! pip install youtube-comment-downloader 

In [2]:
from youtube_comment_downloader import YoutubeCommentDownloader
import pandas as pd
import time 


In [ ]:
#video_id ber korte hobe 
#

In [ ]:
video_ids=[
    "K0B2qlb8eVI",
    "5NLf7O6BF6s"

]
downloader = YoutubeCommentDownloader()
all_data=[]
